Moved this to a separate file (i.e. once OCR is done, I don't need to keep re-running it for tests)

In [ ]:
%pip install -q \
    pandas \
    prisma \
    dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#--schema=../receipts-app/prisma/schema.prisma
# !npm install -g prisma
# !prisma generate

# Setting up prisma python client was pretty hard. For now I copy the schema and .env file from the Next.js app folder
# then source .venv/bin/activate ... and run the prisma commands from there, including installing prisma, generate, and migrate
# !cp ../receipts-app/prisma/dev.db ./prisma/dev.db
# !cp ../receipts-app/prisma/schema.prisma ./prisma/schema.prisma
# !perl -i -pe's/prisma-client-js/prisma-client-py/g' ./prisma/schema.prisma
# !cp ../receipts-app/.env ./.env

In [1]:
import pandas as pd
import asyncio

In [5]:
# grab the CSVs with results from the OCR to save time
receipts_df = pd.read_csv('receipts.csv')
expenses_df = pd.read_csv('expenses.csv')

In [6]:
# # at first I tried with sqlite3 but I might as well use Prisma on both ends

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')
# df1 = pd.read_sql_query("SELECT * from UnitOfMeasure", con)

# print(df.head())

# con.close()

# # using sqlite3 directly

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')

# cur = con.cursor()

# # clear data first
# cur.execute("DELETE FROM UnitOfMeasure")

# # seed to sql lite my data UnitOfMeasure
# for unit in ['g', 'mL', 'Count']:
#     cur.execute("INSERT INTO UnitOfMeasure (name) VALUES (?)", (unit,))

# con.commit()
# con.close()


In [7]:
receipts_df

,box,text,confidence,filename
0,"[[np.int32(631), np.int32(269)], [np.int32(806...",2_3,0.040573,img_4661.jpg
1,"[[np.int32(116), np.int32(263)], [np.int32(821...",I~~fs (pke =,0.124128,img_4661.jpg
2,"[[np.int32(405), np.int32(436)], [np.int32(481...","3 """,0.019708,img_4661.jpg
3,"[[np.int32(120), np.int32(447)], [np.int32(317...",Lec|,0.570547,img_4661.jpg
4,"[[np.int32(656), np.int32(493)], [np.int32(900...",'!,0.086648,img_4661.jpg
...,...,...,...,...
11130,"[[np.int32(464), np.int32(3867)], [np.int32(73...",70104-2707 RTo0o1,0.528192,img_4647.jpg
11131,"[[np.int32(55), np.int32(3892)], [np.int32(501...",THANK You FOR SHOPPING AT 'YIG,0.403911,img_4647.jpg
11132,"[[np.int32(53), np.int32(3931)], [np.int32(259...",STORE MANAGER,0.388632,img_4647.jpg
11133,"[[np.int32(291), np.int32(3929)], [np.int32(53...",SPENCER JONSSON,0.359595,img_4647.jpg


In [7]:
expenses_df

,filename,product,best_match,referenceitem,price
0,img_4659.jpg,7JA),Jam,Jam,NaN
1,img_4588.jpg,CHOCOLAT,Chocolate,Chocolate,6.60
2,img_4601.jpg,Price,Rice,Rice,15.00
3,img_4601.jpg,Not So Classic Chocolate,Chocolate,Chocolate,15.00
4,img_4601.jpg,Mac & Cheese,Mac & Cheese,Mac and cheese,512.99
...,...,...,...,...,...
174,img_4644.jpg,15393 AMY BEAN AND RICE BURRITO GF,Rice,Rice,5.25
175,img_4644.jpg,"29114 LUCKY THAI SPRING ROLLS GF"" 24",Lucky Spring Rolls GF,Spring Rolls,8.65
176,img_4644.jpg,28931 LUCKY SPRING ROLLS GF 241G,Lucky Spring Rolls GF,Spring Rolls,9.25
177,img_4650.jpg,Cooki e,Cookies,Cookies,NaN


In [17]:
# rather do it with prisma, so it's prisma on both sides
# wow, running primsa (npm) with python venv is a ride

import os
from dotenv import load_dotenv
dotenv_path = '../receipts-app/.env'
load_dotenv(dotenv_path)

from prisma import Prisma

async def main():
    # Initialize the Prisma client

    db = Prisma()
    await db.connect()

    # see object attributes (for debug), note that the attributes are in all lowercase here
    # print(dir(db))
    
    # Static data
    unitOfMeasures = pd.DataFrame([
        {'name': 'g'},
        {'name': 'mL'},
        {'name': 'Count'}
    ])
    # # Dynamic data, sample for now
    referenceItems = pd.DataFrame([
        {
            'name': 'Item 1',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 1.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item 2',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 2.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item 3',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 3.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item 4',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 4.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        }
    ])
    receipts = pd.DataFrame([
        {
            'filename': 'IMG_4549.jpg'
        },
        {
            'filename': 'IMG_4550.jpg'
        }
    ])
    receipttexts = pd.DataFrame([
        {'filename': 'IMG_4549.jpg', 'text': 'Item 1', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.95},
        {'filename': 'IMG_4549.jpg', 'text': 'Item 2', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.90},
        {'filename': 'IMG_4550.jpg', 'text': 'Item A', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.93},
        {'filename': 'IMG_4550.jpg', 'text': 'Item B', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.89}
    ])
    # expenses
    expenses = pd.DataFrame([
        {
            # 'receiptTextId': 1, # todo: these need to be specified in the data provided
            # 'referenceItemId': 1 # todo: these need to be specified in the data provided
            'quantiy': 1,
            'priceEach': 1.99, 
        }
    ])

    # create units of measure
    # first clear db
    # try and catch errors
    try:
        await db.referenceitem.delete_many(where={})
        await db.unitofmeasure.delete_many(where={})
        await db.receipt.delete_many(where={})
        print('Db cleared')
    except Exception as e:
        print(e)
    
    for unit in unitOfMeasures.to_dict(orient='records'):
        UoM = await db.unitofmeasure.create(data={'name': unit.get('name')})
        # add UoM.id to unitOfMeasures df once added to db
        unitOfMeasures.loc[unitOfMeasures['name'] == unit['name'], 'unitOfMeasureId'] = UoM.id
        # i could have manually specified foreign keys, but I like to use "remote" key values as a practice during imports
    unitOfMeasures['unitOfMeasureId'] = unitOfMeasures['unitOfMeasureId'].astype(int)
    # print(unitOfMeasures)

    # map unitOfMeasures to referenceItems
    unitOfMeasures_map = unitOfMeasures.set_index('name')['unitOfMeasureId'].to_dict()
    referenceItems['unitOfMeasureId'] = referenceItems['unitOfMeasure'].map(unitOfMeasures_map) #.apply(lambda x: {'connect': {'id': x}})
    referenceItems.drop(columns=['unitOfMeasure'], inplace=True)

    # create reference items
    # for loop with index of referenceItems
    for index, referenceItem in referenceItems.iterrows():
        prisma_data = {
            'name': referenceItem['name'],
            'quantity': referenceItem['quantity'],
            'price': referenceItem['price'],
            'pricePerWeight': referenceItem['pricePerWeight'],
            'referenceUrl': referenceItem['referenceUrl'],
            'unitOfMeasure': {
                'connect': {'id': referenceItem['unitOfMeasureId']}
            }
        }
        # print(f'primsa_data: {prisma_data}')
        db_referenceItem = await db.referenceitem.create(data=prisma_data)
        referenceItems.loc[index, 'referenceItemId'] = db_referenceItem.id
    referenceItems['referenceItemId'] = referenceItems['referenceItemId'].astype(int)
    # print(referenceItems)


    # create receipts
    for index, receipt in receipts.iterrows():
        prisma_data = {
            'filename': receipt['filename']
        }
        # print(f'primsa_data: {prisma_data}')
        db_receipt = await db.receipt.create(data=prisma_data)
        receipts.loc[index, 'receiptId'] = db_receipt.id
    receipts['receiptId'] = receipts['receiptId'].astype(int)
    # print(receipts)

    # map receipts to receipttexts
    receipts_map = receipts.set_index('filename')['receiptId'].to_dict()
    receipttexts['receiptId'] = receipttexts['filename'].map(receipts_map)
    
    # create receipt texts
    for index, receipttext in receipttexts.iterrows():
        prisma_data = {
            'text': receipttext['text'],
            'boundingBox': receipttext['boundingBox'],
            'confidenceScore': receipttext['confidenceScore'],
            'receipt': {
                'connect': {'id': receipttext['receiptId']}
            }
        }
        # print(f'primsa_data: {prisma_data}')
        db_receiptText = await db.receipttext.create(data=prisma_data)
        receipttexts.loc[index, 'receiptTextId'] = db_receiptText.id
    receipttexts['receiptTextId'] = receipttexts['receiptTextId'].astype(int)
    print(receipttexts)

    # create expenses
    # model Expense {
    #   id        Int   @id @default(autoincrement())
    #   priceEach Float
    #   quantity  Float

    #   receiptText     ReceiptText   @relation(fields: [receiptTextId], references: [id])
    #   receiptTextId   Int
    #   referenceItemId Int
    #   referenceItem   ReferenceItem @relation(fields: [referenceItemId], references: [id])

    #   createdAt DateTime @default(now())
    #   updatedAt DateTime @updatedAt
    # } 
    for index, expense in expenses.iterrows():
        prisma_data = {
            'priceEach': expense['priceEach'],
            'quantity': 1, # known shortcoming here, I didn't implement finding the quantity from the OCR, will need to enter manually
            'receiptText': {
                'connect': {'id': receipttexts.loc[receipttexts['filename'] == expense['filename'], 'receiptTextId'].values[0]}
            },
            'referenceItem': {
                'connect': {'id': referenceItems.loc[referenceItems['name'] == expense['best_match'], 'referenceItemId'].values[0]}
            }
        }
        # print(f'primsa_data: {prisma_data}')
        db_expense = await db.expense.create(data=prisma_data)
        expenses.loc[index, 'expenseId'] = db_expense.id


    # # test create expense
    # expense = await db.expense.create(
    #     data={
    #         'filename': 'IMG_4549.jpg',
    #         'priceEach': 1.99,
    #         'quantity': 1,
    #         ...

    # # create all the reference items
    # for referenceItem in referenceItems:
    #     await db.referenceItem.create(data=referenceItem)

    # Iterate through the receipts and create them along with their texts
    # for receipt_data in receipts:
    #     # Create the Receipt
    #     receipt = await db.receipt.create(
    #         data={
    #             'filename': receipt_data['filename'],
    #             'receiptText': {
    #                 'create': receipt_data['receiptTexts']
    #             }
    #         },
    #         include={
    #             'receiptText': True
    #         }
    #     )
    #     print(f'Receipt created: {receipt.id}, with texts: {[text.text for text in receipt.receiptText]}')

    # Disconnect the Prisma client
    await db.disconnect()

# Run the main function
# asyncio.run(main()) # not for jupyter
await main()

Foreign key constraint failed on the field: `foreign key`
       filename    text                             boundingBox  \
0  IMG_4549.jpg  Item 1   {"x":0,"y":0,"width":100,"height":20}   
1  IMG_4549.jpg  Item 2  {"x":0,"y":20,"width":100,"height":20}   
2  IMG_4550.jpg  Item A   {"x":0,"y":0,"width":100,"height":20}   
3  IMG_4550.jpg  Item B  {"x":0,"y":20,"width":100,"height":20}   

   confidenceScore  receiptId  receiptTextId  
0             0.95         30             49  
1             0.90         30             50  
2             0.93         31             51  
3             0.89         31             52  


KeyError: 'filename'

In [14]:
# # update Next.js app
# !cp ./prisma/dev.db ../receipts-app/prisma/dev.db
# !cp ./prisma/schema.prisma ../receipts-app/prisma/schema.prisma
# !perl -i -pe's/prisma-client-py/prisma-client-js/g' ../receipts-app/prisma/schema.prisma
# !cp ./.env ../receipts-app/.env